In [2]:
!sudo pip install 'syft[udacity]'

  Using cached https://files.pythonhosted.org/packages/ba/f1/f1d00ca775eb15bbeb3c5eba85ef4473005adb03e745f7d2c10d45524831/syft-0.2.6.tar.gz
  Using cached https://files.pythonhosted.org/packages/f2/28/2a03252dfb9ebf377f40fba6a7841b47083260bf8bd8e737b0c6952df83f/Flask-1.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/ad/61f8dfba88c4e56196bf6d056cdbba64dc9c5dfdfbc97d02e6472feed913/Pillow-6.2.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/1a/34/d9c8e19b4d5157721a5b77750116c6bb6355f1d85b92e7de491269b9ee51/aiortc-0.9.28.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-Bl4Byu/aiortc/setup.py", line 9, in <module>
        with open(about_file, encoding="utf-8") as fp:
    TypeError: 'encoding' is an invalid keyword argument for this function
    
    ----------------------------------------


## Adopted Toy model from
+ https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2012%20-%20Train%20an%20Encrypted%20Neural%20Network%20on%20Encrypted%20Data.ipynb
+ SMPC and homomorphic encryption provide mechanisms to enable computation on encrypted data, without decrypting the underlying values themselves. As a result, data remains encrypted in memory, in process and at rest.
+ https://baffle.io/blog/homomorphic-and-multiparty-computation/

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [73]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [77]:
x = torch.tensor([25])
x

tensor([25])

In [78]:
encrypted_x = x.share(bob, alice, james)
encrypted_x

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:75382377266 -> bob:15890766040]
	-> [PointerTensor | me:32947143687 -> alice:1167520626]
	-> [PointerTensor | me:9749565363 -> james:87315513379]
	*crypto provider: me*

In [79]:
encrypted_x.get()

tensor([25])

In [80]:
project_id = 'covid-19-271622'

In [81]:
%%bigquery --project $project_id 
SELECT result_quality, Allele, 
AEAD.DECRYPT_STRING((SELECT keyset FROM `encryption.encryption_key` as keys where keys.ligand_id=mhc.ligand_id), 
mhc.A1_encrypted, CAST(mhc.ligand_id AS STRING)) A1_decrypted, 
AEAD.DECRYPT_STRING((SELECT keyset FROM `encryption.encryption_key` as keys where keys.ligand_id=mhc.ligand_id), 
mhc.A2_encrypted, CAST(mhc.ligand_id AS STRING)) A2_decrypted, 
A3, A4, A5, A6, A7, A8, A9, A10 
FROM `corona.mhc_qual_feature_encryption` as mhc
WHERE ligand_id = 1717014 OR ligand_id = 1025027

,result_quality,Allele,A1_decrypted,A2_decrypted,A3,A4,A5,A6,A7,A8,A9,A10
0,1,HLA-A*02:01,0,0,0,0,0,0,0,0,0,0
1,0,HLA-A*68:01,1,1,5,1,13,11,11,20,12,0


In [82]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

In [83]:
print(data)

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])


In [84]:
print(target)

tensor([[0.],
        [0.],
        [1.],
        [1.]])


In [85]:
print(model)

Net(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)


In [86]:
# none-encrypted list(bob._tensors.values())

In [87]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

/opt/conda/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:78: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")
/opt/conda/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:90: UserWarning: Default args selected
  warnings.warn("Default args selected")


In [88]:
print(data)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:25737328515 -> bob:23320114442]
	-> [PointerTensor | me:64819730123 -> alice:50650138937]
	*crypto provider: james*


In [89]:
print(data)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:25737328515 -> bob:23320114442]
	-> [PointerTensor | me:64819730123 -> alice:50650138937]
	*crypto provider: james*


In [90]:
print(target)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:34747258875 -> bob:83402176022]
	-> [PointerTensor | me:84918744592 -> alice:14118335382]
	*crypto provider: james*


In [91]:
print(model(data))

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:93093756967 -> bob:39277013840]
	-> [PointerTensor | me:93578185764 -> alice:88575155504]
	*crypto provider: james*


In [92]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

tensor(2.2960)
tensor(1.1550)
tensor(0.6120)
tensor(0.2710)
tensor(0.1550)
tensor(0.0880)
tensor(0.0540)
tensor(0.0330)
tensor(0.0220)
tensor(0.0150)
tensor(0.0110)
tensor(0.0090)
tensor(0.0070)
tensor(0.0060)
tensor(0.0070)
tensor(0.0040)
tensor(0.0050)
tensor(0.0030)
tensor(0.0040)
tensor(0.0030)


## Adopted Model from
+ https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2011%20-%20Secure%20Deep%20Learning%20Classification.ipynb
+ Might need to install -> !pip install torchvision 

+ Might need to install -> pip install pillow

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [4]:
epochs = 10
n_test_batches = 200

In [5]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/jupyter/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

In [7]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/jupyter/data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
    ))

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [9]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.301686
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.209247
Train Epoch: 1 [12800/60032 (21%)]	Loss: 0.247478
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.187667
Train Epoch: 1 [25600/60032 (43%)]	Loss: 0.124999
Train Epoch: 1 [32000/60032 (53%)]	Loss: 0.091852
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.104337
Train Epoch: 1 [44800/60032 (75%)]	Loss: 0.057239
Train Epoch: 1 [51200/60032 (85%)]	Loss: 0.133403
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.107137
Train Epoch: 2 [0/60032 (0%)]	Loss: 0.110678
Train Epoch: 2 [6400/60032 (11%)]	Loss: 0.023410
Train Epoch: 2 [12800/60032 (21%)]	Loss: 0.121223
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.411033
Train Epoch: 2 [25600/60032 (43%)]	Loss: 0.062868
Train Epoch: 2 [32000/60032 (53%)]	Loss: 0.055777
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.079457
Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.034083
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.029965
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.037514
Train Epoch: